#f(x,y) = x^2y + y + 2
* Makes use of multiple cores
* tensor means similar to numpy

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(3,name='x')

In [4]:
y = tf.Variable(4,name='y')

In [5]:
f = x*x*y + y + 2

In [6]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [7]:
#init all variables
init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    init.run() # execute all inits
    result = f.eval()

In [9]:
result

42

In [11]:
# Create a separate session
graph = tf.Graph()

In [15]:
#inside this scope, graph will be treated as default
with graph.as_default():
    x2 = tf.Variable(2)

In [16]:
x2.graph is graph

True

In [17]:
x2.graph is tf.get_default_graph()

False

### Linear Regression with TensorFlow

In [18]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [19]:
housing = fetch_california_housing()

In [22]:
m,n = housing.data.shape

In [23]:
m # num. of rows of housing information

20640

In [25]:
n # num. of features

8

In [26]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [29]:
X = tf.constant(housing.data, dtype=tf.float32 ,name="X")

In [30]:
Y = tf.constant(housing.target, dtype=tf.float32, name="y")

In [31]:
housing.target

array([ 4.526,  3.585,  3.521, ...,  0.923,  0.847,  0.894])

In [33]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ..., 
        [   1.7       ,   17.        ,    

In [34]:
XT = tf.transpose(X)

In [35]:
X.shape

TensorShape([Dimension(20640), Dimension(8)])

In [36]:
Y.shape

TensorShape([Dimension(20640)])

In [42]:
M1 = tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT)

In [43]:
M1.shape

TensorShape([Dimension(8), Dimension(20640)])

In [45]:
Y

<tf.Tensor 'y_1:0' shape=(20640,) dtype=float32>

In [46]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [47]:
tf.reset_default_graph()

X = tf.constant(housing_data_plus_bias, dtype=tf.float64, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float64, name="y")
XT = tf.transpose(X)

In [48]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [50]:
housing_data_plus_bias.shape

(20640, 9)

In [51]:
housing.target.reshape(-1, 1).shape

(20640, 1)

In [52]:
theta.shape

TensorShape([Dimension(9), Dimension(1)])

In [53]:
with tf.Session() as sess:
    result = theta.eval()

In [54]:
result

array([[ -3.69419202e+01],
       [  4.36693293e-01],
       [  9.43577803e-03],
       [ -1.07322041e-01],
       [  6.45065694e-01],
       [ -3.97638942e-06],
       [ -3.78654265e-03],
       [ -4.21314378e-01],
       [ -4.34513755e-01]])

In [55]:
### Saving & restoring a model

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [64]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "D:\Data-Science-ML\data-science\data\my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "my_model_final.ckpt")

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.75443
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.57278
Epoch 300 MSE = 0.558501
Epoch 400 MSE = 0.549069
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.531243
Epoch 900 MSE = 0.529371


ValueError: Parent directory of my_model_final.ckpt doesn't exist, can't save.